In [33]:
from CRF import CRF
from utils import crf_train_loop
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.optim import Adam

In [2]:
# two dice one is fair, one is loaded
fair_dice = np.array([1/6]*6)
loaded_dice = np.array([0.04,0.04,0.04,0.04,0.04,0.8])

probabilities = {'fair': fair_dice,
                'loaded': loaded_dice}

In [3]:
# if dice is fair at time t, 0.6 chance we stay fair, 0.4 chance it is loaded at time 2
transition_mat = {'fair': np.array([0.6, 0.4, 0.0]),
                 'loaded': np.array([0.3, 0.7, 0.0]),
                 'start': np.array([0.5, 0.5, 0.0])}
states = ['fair', 'loaded', 'start']
state2ix = {'fair': 0,
           'loaded': 1,
           'start': 2}

log_likelihood = np.hstack([np.log(fair_dice).reshape(-1,1), 
                            np.log(loaded_dice).reshape(-1,1)])

In [4]:
def simulate_data(n_timesteps):
    data = np.zeros(n_timesteps)
    prev_state = 'start'
    state_list = np.zeros(n_timesteps)
    for n in range(n_timesteps):
        next_state = np.random.choice(states, p=transition_mat[prev_state])
        state_list[n] = state2ix[next_state]
        next_data = np.random.choice([0,1,2,3,4,5], p=probabilities[next_state])
        data[n] = next_data
        prev_state = next_state
    return data, state_list

In [5]:
n_obs = 15
rolls = np.zeros((5000, n_obs)).astype(int)
targets = np.zeros((5000, n_obs)).astype(int)

for i in range(5000):
    data, dices = simulate_data(n_obs)
    rolls[i] = data.reshape(1, -1).astype(int)
    targets[i] = dices.reshape(1, -1).astype(int)

In [41]:
print(torch.nn.init.normal_(nn.Parameter(torch.randn(n_dice, n_dice + 1)), -1, 0.1))

Parameter containing:
tensor([[-1.1438, -1.0999, -0.9955],
        [-1.0071, -1.1127, -0.9919]], requires_grad=True)


In [99]:
def crf_train_loop(model, rolls, targets, n_epochs, learning_rate=0.01):

    optimizer = Adam(model.parameters(), lr=learning_rate,
                     weight_decay=1e-4)

    for epoch in range(n_epochs):
        batch_loss = []
        N = rolls.shape[0]
        model.zero_grad()
        for index, (roll, labels) in enumerate(zip(rolls, targets)):
            # Forward Pass
            neg_log_likelihood = model.neg_log_likelihood(roll, labels)
            batch_loss.append(neg_log_likelihood)
            
            if index % 50 == 0:
                ll = torch.cat(batch_loss).mean()
                ll.backward()
                optimizer.step()
                print("Epoch {}: Batch {}/{} loss is {:.4f}".format(epoch, index//50,N//50,ll.data.numpy()))
                batch_loss = []
    
    return model

In [103]:
"""Inspired by http://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html"""
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.optim import Adam
import numpy as np


class CRF(nn.Module):
    
    def __init__(self, n_dice, log_likelihood):
        super(CRF, self).__init__()
        
        self.n_states = n_dice
        self.transition = torch.nn.init.normal_(nn.Parameter(torch.randn(n_dice, n_dice + 1)), -1, 0.1)
        self.loglikelihood = log_likelihood
    

    def to_scalar(self, var):
        return var.view(-1).data.tolist()[0]


    def argmax(self, vec):
        _, idx = torch.max(vec, 1)
        return self.to_scalar(idx)
        
    # numerically stable log sum exp
    # Source: http://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html
    def log_sum_exp(self, vec):
        max_score = vec[0, self.argmax(vec)]
        max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
        return max_score + \
               torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))
    
    
    def _data_to_likelihood(self, rolls):
        """Converts a numpy array of rolls (integers) to log-likelihood.
        Input is one [1, n_rolls]
        """
        return Variable(torch.FloatTensor(self.loglikelihood[rolls]), requires_grad=False)
        
    
    def _compute_likelihood_numerator(self, loglikelihoods, states):
        """Computes numerator of likelihood function for a given sequence.
        
        We'll iterate over the sequence of states and compute the sum 
        of the relevant transition cost with the log likelihood of the observed
        roll. 
        Input:
            loglikelihoods: torch Variable. Matrix of n_obs x n_states. 
                            i,j entry is loglikelihood of observing roll i given state j
            states: sequence of labels
        Output:
            score: torch Variable. Score of assignment. 
        """
        prev_state = self.n_states
        score = Variable(torch.Tensor([0]))
        for index, state in enumerate(states):
            score += self.transition[state, prev_state] + loglikelihoods[index, state]
            prev_state = state
        return score
    
    def _compute_likelihood_denominator(self, loglikelihoods):
        """Implements the forward pass of the forward-backward algorithm.
        
        We loop over all possible states efficiently using the recursive
        relationship: alpha_t(j) = \sum_i alpha_{t-1}(i) * L(x_t | y_t) * C(y_t | y{t-1} = i)
        Input:
            loglikelihoods: torch Variable. Same input as _compute_likelihood_numerator.
                            This algorithm efficiently loops over all possible state sequences
                            so no other imput is needed.
        Output:
            torch Variable. 
        """

        # Stores the current value of alpha at timestep t
        prev_alpha = self.transition[:, self.n_states] + loglikelihoods[0].view(1, -1)

        for roll in loglikelihoods[1:]:
            alpha_t = []

            # Loop over all possible states
            for next_state in range(self.n_states):
                
                # Compute all possible costs of transitioning to next_state
                feature_function = self.transition[next_state,:self.n_states].view(1, -1) +\
                                   roll[next_state].view(1, -1).expand(1, self.n_states)
                
#                 print(feature_function.shape)
                alpha_t_next_state = prev_alpha + feature_function
#                 print(alpha_t_next_state.shape)
                x = self.log_sum_exp(alpha_t_next_state).view(1)
#                 print('X: ')
#                 print(x)
#                 print(x.shape)
                alpha_t.append(self.log_sum_exp(alpha_t_next_state).view(1))
#                 print(len(alpha_t))
            
#             print(alpha_t)
#             print(len(alpha_t))
#             print('First:')
#             print(alpha_t[0])
#             print(alpha_t[0].shape)
#             print('Second:')
#             print(alpha_t[1])
#             print(alpha_t[1].shape)
            prev_alpha = torch.cat(alpha_t).view(1, -1)
#         print(self.log_sum_exp(prev_alpha).view(1).shape)
        return self.log_sum_exp(prev_alpha)
    
    def _viterbi_algorithm(self, loglikelihoods):
        """Implements Viterbi algorithm for finding most likely sequence of labels.
        
        Very similar to _compute_likelihood_denominator but now we take the maximum
        over the previous states as opposed to the sum. 
        Input:
            loglikelihoods: torch Variable. Same input as _compute_likelihood_denominator.
        Output:
            tuple. First entry is the most likely sequence of labels. Second is
                   the loglikelihood of this sequence. 
        """

        argmaxes = []

        # prev_delta will store the current score of the sequence for each state
        prev_delta = self.transition[:, self.n_states].contiguous().view(1, -1) +\
                      loglikelihoods[0].view(1, -1)

        for roll in loglikelihoods[1:]:
            local_argmaxes = []
            next_delta = []
            for next_state in range(self.n_states):
                feature_function = self.transition[next_state,:self.n_states].view(1, -1) +\
                                   roll.view(1, -1) +\
                                   prev_delta
                most_likely_state = self.argmax(feature_function)
                score = feature_function[0][most_likely_state]
                next_delta.append(score)
                local_argmaxes.append(most_likely_state)
            prev_delta = torch.cat(next_delta).view(1, -1)
            argmaxes.append(local_argmaxes)
        
        final_state = self.argmax(prev_delta)
        final_score = prev_delta[0][final_state]
        path_list = [final_state]

        # Backtrack through the argmaxes to find most likely state
        for states in reversed(argmaxes):
            final_state = states[final_state]
            path_list.append(final_state)
        
        return np.array(path_list), final_score
        
    def neg_log_likelihood(self, rolls, states):
        """Compute neg log-likelihood for a given sequence.
        
        Input: 
            rolls: numpy array, dim [1, n_rolls]. Integer 0-5 showing value on dice.
            states: numpy array, dim [1, n_rolls]. Integer 0, 1. 0 if dice is fair.
        """
        loglikelihoods = self._data_to_likelihood(rolls)
        states = torch.LongTensor(states)
        
        sequence_loglik = self._compute_likelihood_numerator(loglikelihoods, states)
#         print('Numerator: ')
#         print(sequence_loglik.shape)
#         print('Numerator done')
        denominator = self._compute_likelihood_denominator(loglikelihoods)
        return denominator - sequence_loglik
               
    
    def forward(self, rolls):
        loglikelihoods = self._data_to_likelihood(rolls)
        return self._viterbi_algorithm(loglikelihoods)
                
    

In [104]:
model = CRF(2, log_likelihood)
n_dice=2

In [105]:
model = crf_train_loop(model, rolls, targets, 1, 0.001)

Epoch 0: Batch 0/100 loss is 9.1227
Epoch 0: Batch 1/100 loss is 6.8983
Epoch 0: Batch 2/100 loss is 7.4736
Epoch 0: Batch 3/100 loss is 7.2539
Epoch 0: Batch 4/100 loss is 7.6379
Epoch 0: Batch 5/100 loss is 6.9648
Epoch 0: Batch 6/100 loss is 6.7741
Epoch 0: Batch 7/100 loss is 7.0387
Epoch 0: Batch 8/100 loss is 7.1012
Epoch 0: Batch 9/100 loss is 6.5904
Epoch 0: Batch 10/100 loss is 6.8525
Epoch 0: Batch 11/100 loss is 6.6339
Epoch 0: Batch 12/100 loss is 7.3245
Epoch 0: Batch 13/100 loss is 6.8953
Epoch 0: Batch 14/100 loss is 6.6914
Epoch 0: Batch 15/100 loss is 7.1151
Epoch 0: Batch 16/100 loss is 7.0650
Epoch 0: Batch 17/100 loss is 6.7962
Epoch 0: Batch 18/100 loss is 6.7170
Epoch 0: Batch 19/100 loss is 7.1804
Epoch 0: Batch 20/100 loss is 6.9839
Epoch 0: Batch 21/100 loss is 7.2463
Epoch 0: Batch 22/100 loss is 6.6751
Epoch 0: Batch 23/100 loss is 6.9411
Epoch 0: Batch 24/100 loss is 7.4206
Epoch 0: Batch 25/100 loss is 6.4920
Epoch 0: Batch 26/100 loss is 6.8778
Epoch 0: Ba

In [ ]:
torch.save(model.state_dict(), "./checkpoint.hdf5")

In [ ]:
model.load_state_dict(torch.load("./checkpoint.hdf5"))

In [ ]:
data, dices = simulate_data(15)
test_rolls = data.reshape(1, -1).astype(int)
test_targets = dices.reshape(1, -1).astype(int)

In [ ]:
test_rolls[0]

In [ ]:
model.forward(test_rolls[0])[0]

In [9]:
test_targets[0]

NameError: name 'test_targets' is not defined

In [ ]:
list(model.parameters())[0].data.numpy()

In [ ]:
data, dices = simulate_data(15)
test_rolls = data.reshape(1, -1).astype(int)
test_targets = dices.reshape(1, -1).astype(int)

In [ ]:
test_rolls[0]

In [ ]:
model.forward(test_rolls[0])[0]

In [ ]:
test_targets[0]